In [ ]:
from six.moves import urllib
import json
import os
import datetime as dt

import pandas as pd

# copy pasted from: https://github.com/prisma-labs/python-graphql-client/blob/master/graphqlclient/client.py
class GraphQLClient:
    def __init__(self, endpoint):
        self.endpoint = endpoint
        self.token = None
        self.headername = None

    def execute(self, query, variables=None):
        return self._send(query, variables)

    def inject_token(self, token, headername='token'):
        self.token = token
        self.headername = headername

    def _send(self, query, variables):
        data = {'query': query,
                'variables': variables}
        headers = {'Accept': 'application/json',
                   'Content-Type': 'application/json'}

        if self.token is not None:
            headers[self.headername] = '{}'.format(self.token)

        req = urllib.request.Request(self.endpoint, json.dumps(data).encode('utf-8'), headers)

        try:
            response = urllib.request.urlopen(req)
            return response.read().decode('utf-8')
        except urllib.error.HTTPError as e:
            print((e.read()))
            print('')
            raise e

def get_part(client, part, manufacturer):
    dsl = '''
    query ($info: MpnInput!) {
      part(mpn: $info) {
        mpn {
          manufacturer
          part
        }
        type
        datasheet
        description
        image {
          url
          credit_string
          credit_url
        }
        specs {
          key
          name
          value
        }
        offers {
          sku {
            vendor
            part
          }
          description
          moq
          in_stock_quantity
          stock_location
          image {
            url
            credit_string
            credit_url
          }
          specs {
            key
            name
            value
          }
          prices {
            GBP
            EUR
            USD
          }
        }
      }
    }
    '''
    resp = client.execute(dsl, {'info': {'part': part, 'manufacturer': manufacturer}})
    return json.loads(resp)['data']['part']

def get_manufacturers(client, mpn):
    dsl = '''
    query SomeParts($q: String!) {
      search(term: $q) {
            mpn {
          manufacturer
          part
        }

      }
    }
    '''
    resp = client.execute(dsl, {'q': mpn})
    return [row['mpn'] for row in json.loads(resp)['data']['search'] if row['mpn']['part'] == mpn]

In [ ]:
# Call the kitspace partinfo API

client = GraphQLClient('https://dev-partinfo.kitspace.org/graphql')
part = get_part(client, part='NE555P', manufacturer='Texas Instruments')
part

{'mpn': {'manufacturer': 'Texas Instruments', 'part': 'NE555P'},
 'type': 'match',
 'datasheet': 'https://octopart.com/click/track?ai=8078&ct=datasheets&sig=0974045&at=physicalpart&sid=370&ppid=55400998&hlid=30046660',
 'description': 'Texas Instruments NE555PTIMER Single Precision, DIP8 , 0.5MHZ',
 'image': {'url': 'https://sigma.octopart.com/78379198/image/Texas-Instruments-NE555P-..jpg',
  'credit_string': 'element14 APAC',
  'credit_url': 'http://www.element14.com/'},
 'specs': [{'key': 'case_package', 'name': 'Case/Package', 'value': 'PDIP'},
  {'key': 'pin_count', 'name': 'Number of Pins', 'value': '8'},
  {'key': 'lead_free_status',
   'name': 'Lead-Free Status',
   'value': 'Lead Free'},
  {'key': 'current_rating', 'name': 'Current Rating', 'value': '10.0 A'},
  {'key': 'packaging', 'name': 'Packaging', 'value': 'Tube'},
  {'key': 'lifecycle_status', 'name': 'Lifecycle Status', 'value': 'Active'},
  {'key': 'rohs_status', 'name': 'RoHS', 'value': 'Compliant'},
  {'key': 'supply

In [ ]:
# Convert stock levels to a dataframe

def part_stock_df(part):
    stock = {
        "Manufacturer": part["mpn"]["manufacturer"],
        "MPN": part["mpn"]["part"]
    }
    stock.update({f"{offer['sku']['vendor']}": offer['in_stock_quantity'] for offer in part['offers']})
    return pd.DataFrame(stock, index=[dt.datetime.utcnow()])

df = part_stock_df(part)
df

,Manufacturer,MPN,Digikey,Mouser,Newark,Farnell
2021-10-27 23:27:44.891884,Texas Instruments,NE555P,0,1910,21350,0


In [ ]:
# Load BOM file

df = pd.read_csv("../manufacturing/40-channel-hv-switching-board_BOM.csv")
df

,Refs,quantity,Manufacturer,Mfg_Part_No,datasheet,footprint,value
0,"C1, C3, C5, C6, C8-C13, C16",11,Samsung,CL21B104KBCNNNC,NaN,Capacitors_SMD:C_0805,0.1uF
1,"C14, C15",2,Samsung,CL21C151JBANNNC,NaN,Capacitors_SMD:C_0805,150pF
2,"C2, C4",2,KEMET,T491D336K020AT,NaN,Sci-Bots:SM2917,33uF
3,C7,1,Samsung,CL21B103KCANNNC,NaN,Capacitors_SMD:C_0805,0.01uF
4,DS1,1,Würth Elektronik,150080BS75000,NaN,LEDs:LED_0805,+3.3V_PWR
5,FB1-FB5,5,Würth Elektronik,742792040,NaN,Resistors_SMD:R_0805,FERRITE
6,JP1,1,NaN,NaN,NaN,Resistors_SMD:R_0805,JUMPER
7,P1,1,NaN,NaN,NaN,Pin_Headers:Pin_Header_Angled_1x06,CONN_01X06
8,P2,1,NaN,NaN,NaN,Pin_Headers:Pin_Header_Straight_2x03,CONN_01X06
9,P3-P6,4,NaN,NaN,NaN,Connect:1pin,CONN_01X01


In [ ]:
# Lookup manufacturers on Octopart (based on MPN) and save to BOM

if 'Manufacturer' not in df.columns:
    manufacturers = [""] * len(df)
    for i, part in df.iterrows():
        if type(part["MPN"]) == str:
            name = get_manufacturers(client, part["MPN"])[0]['manufacturer']
            print(name)
            manufacturers[i] = name
    df["Manufacturer"] = manufacturers
    df.to_csv("../40-channel-hv-switching-board_BOM.csv", index=False)

In [ ]:
df_stock = pd.DataFrame()

# Iterate over all parts that have a MPN
for i, part in df.iloc[df["Mfg_Part_No"].dropna().index].iterrows():
    df_stock = df_stock.append(part_stock_df(get_part(client, part=part['Mfg_Part_No'], manufacturer=part['Manufacturer'])))
    
df_stock

0      CL21B104KBCNNNC
1      CL21C151JBANNNC
2       T491D336K020AT
3      CL21B103KCANNNC
4        150080BS75000
5            742792040
6                  NaN
7                  NaN
8                  NaN
9                  NaN
10     TFM-125-02-L-DH
11     RC0805FR-0751RL
12    RC0805FR-07150RL
13     RC0805FR-0710KL
14      RC0805FR-071KL
15         MM74HC595MX
16     W25Q64FVSSIG TR
17       ATMEGA328P-AU
18     CSTCE8M00G55-R0
Name: Mfg_Part_No, dtype: object
0      CL21B104KBCNNNC
1      CL21C151JBANNNC
2       T491D336K020AT
3      CL21B103KCANNNC
4        150080BS75000
5            742792040
6                  NaN
7                  NaN
8                  NaN
9                  NaN
10     TFM-125-02-L-DH
11     RC0805FR-0751RL
12    RC0805FR-07150RL
13     RC0805FR-0710KL
14      RC0805FR-071KL
15         MM74HC595MX
16     W25Q64FVSSIG TR
17       ATMEGA328P-AU
18     CSTCE8M00G55-R0
Name: Mfg_Part_No, dtype: object
0      CL21B104KBCNNNC
1      CL21C151JBANNNC
2       T491D3

,Manufacturer,MPN,RS,Farnell,Digikey,Newark,Mouser
2021-10-28 13:19:41.893643,Samsung,CL21B104KBCNNNC,4452000.0,480000.0,0.0,284000.0,938349.0
2021-10-28 13:19:42.334640,Samsung,CL21C151JBANNNC,NaN,NaN,0.0,NaN,NaN
2021-10-28 13:19:42.776969,KEMET,T491D336K020AT,NaN,0.0,0.0,0.0,8.0
2021-10-28 13:19:43.281970,Samsung,CL21B103KCANNNC,NaN,4054.0,43025.0,84108.0,0.0
2021-10-28 13:19:43.716970,Würth Elektronik,150080BS75000,11250.0,2905.0,46022.0,2846.0,14077.0
2021-10-28 13:19:44.168572,Würth Elektronik,742792040,45400.0,30831.0,68754.0,33033.0,96499.0
2021-10-28 13:19:44.598204,Samtec,TFM-125-02-L-DH,8.0,2.0,1229.0,2.0,90.0
2021-10-28 13:19:45.021736,Yageo,RC0805FR-0751RL,30000.0,217.0,1399098.0,217.0,0.0
2021-10-28 13:19:45.563738,Yageo,RC0805FR-07150RL,NaN,0.0,0.0,0.0,21627.0
2021-10-28 13:19:45.986739,Yageo,RC0805FR-0710KL,520000.0,0.0,6035125.0,1310000.0,0.0


In [ ]:
# Get unique Manufacturer/MPN pairs

import numpy as np

for manufacturer, mpn in df_stock.groupby(['Manufacturer', 'MPN']).count().index.values:
    df_stock[np.logical_and(df_stock['Manufacturer'] == manufacturer , df_stock['MPN'] == mpn)]

In [ ]:
df_stock[np.logical_and(df_stock['Manufacturer'] == manufacturer , df_stock['MPN'] == mpn)]

,Manufacturer,MPN,RS,Farnell,Digikey,Newark,Mouser
2021-10-28 13:19:46.853897,onsemi,MM74HC595MX,3715.0,204.0,0.0,0.0,0.0
